# Preparing the dataset
Objective: Try a second approach for the images that didn't work. 

In [219]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import os 


def save_image(name, img):
    cv.imwrite('..\datasets\ISIC2017\Training_data_processed\\not_successfully_cutted\\{}.jpg'.format(name), img)

def save_in_test(name, img):
    cv.imwrite('..\datasets\ISIC2017\Training_data_processed\\tests\\{}.jpg'.format(name), img)

def read_image(name):
    img = cv.imread('..\datasets\ISIC2017\Training_data_processed\\not_successfully_cutted\\{}.jpg'.format(name))
    return img

all_images = []

Obtaining all images name

In [220]:
all_files = os.listdir('..\datasets\ISIC2017\Training_data_processed\\not_successfully_cutted\\')

for file in all_files:
        if file.find('superpixels') == -1 and file.find('Training_Data_metadata') == -1:
                new_img = file.split('.')[0]
                all_images.append(new_img)


Use morphological operations to better binarize the image

In [221]:
def morphological_operations(img):
    kernel = np.ones((10,10),np.uint8)
    img = cv.dilate(img,kernel,iterations = 2) 
    img = cv.erode(img,kernel,iterations = 2) 
    return img

In [222]:
# img_original = read_image('ISIC_0014395')
# img = morphological_operations(img_original)
# save_in_test('ISIC_0014395', img_original)
# save_in_test('ISIC_0014395_test', img)


Obtain the threshold image for gray scale image and blue channel

In [223]:
def threshold_image (img):
    img = cv.GaussianBlur(img,(5,5),0)
    _,img = cv.threshold(img,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
    kernel = np.ones((10,10),np.uint8)
    img = cv.erode(img,kernel,iterations = 1) 
    return img

Obtain the threshold image to highlight the black corners

In [224]:
def highlight_black_corners (img):
    img = cv.GaussianBlur(img,(5,5),0)
    y, x = img.shape
    for i in range(5):
        img[i,:] = 0
        img[:,i] = 0
        img[:,x-i-1] = 0
        img[y-i-1,:] = 0
    _,img = cv.threshold(img,15,255,cv.THRESH_BINARY)
    return img

Remove black corners remaining from the lens of the camera used to take the lesion image 

In [225]:
def remove_corners(img):
    y, x = img.shape

    corners = {
        'top_left': {
            'x': [0,10],
            'y': [0,10]
        },
        'top_right': {
            'x': [0,10],
            'y': [y - 11, y-1]
        },
        'bottom_left': {
            'x': [x - 11, x-1],
            'y': [0,10]
        },
        'bottom_right': {
            'x': [x - 11, x-1],
            'y': [y - 11, y-1]
        },
    }

    for position in corners:
        for x in range(corners[position]['x'][0], corners[position]['x'][1]):
            for y in range(corners[position]['y'][0], corners[position]['y'][1]):
                if img[y,x] >= 255:
                    cv.floodFill(img, None, (x,y), 0)
    return img

Add gray and blue images

In [226]:
def add_gray_and_blue(img_1, img_2):
    img_sum = img_1 + img_2
    img_sum[img_sum > 125] = 255
    img_sum[img_sum < 125] = 0
    return img_sum
    

Remove minimal blobs from the image

In [227]:
def remove_minimal_blobs(img):

    kernel = np.ones((10,10),np.uint8)
    img_clean = cv.erode(img,kernel,iterations = 2)

    contours, _ = cv.findContours(img_clean, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    img_clean = np.zeros_like(img)
    for contour in contours:
        area = cv.contourArea(contour)
        if area > 8000:
            cv.fillPoly(img_clean, [contour], 255)


    return img_clean

Get coordinates of rectangle around ROI

In [228]:
def get_coordinates(img, gap):
    contours, _ = cv.findContours(img,cv.RETR_TREE,cv.CHAIN_APPROX_NONE)

    if len(contours) == 0:
        return []

    y, x = img.shape
    coords = []
    
    t, b, r, l = [-1, 9999999, -1, 9999999]
    
    for points in contours:
        for p in points:
            if t < p[0][1]:
                t = p[0][1]
            if b > p[0][1]:
                b = p[0][1]
            if l > p[0][0]:
                l = p[0][0]
            if r < p[0][0]:
                r = p[0][0]

    coords.append(t + gap if t + gap < y else y)
    coords.append(b - gap if b - gap > -1 else 0)
    coords.append(r + gap if r + gap < x else x)
    coords.append(l - gap if l - gap > -1 else 0)

    return coords

Draw rectangle based on found coordinates

In [229]:
def draw_rectangle(img, coords):

    [t,b,r,l] = coords
                
    img_final = cv.rectangle(img, (l,b), (r,t), (255, 0, 0), 3)

    return img_final

Crop in coordinates     

In [230]:
def crop_image(img, coords):
    if len(coords) != 4:
        return img
    
    [t,b,r,l] = coords
    crop = img[b:t, l:r, :]

    return crop

Get maximized coordinates

In [231]:
def get_max_coords(max_coords, coords):
    if len(coords) != 4:
        return max_coords

    [t_m,b_m,r_m,l_m] = max_coords
    [t,b,r,l] = coords

    t_m = t if t_m > t else t_m
    b_m = b if b_m < b else b_m
    r_m = r if r_m < r else r_m
    l_m = l if l_m > l else l_m

    return [t_m,b_m,r_m,l_m]

In [236]:
max_coords = [-1, 9999999, -1, 9999999]
img_name = ''
all_images = ["ISIC_0000036"]
try:
    for img_name in all_images:
        img_rgb = read_image(img_name)
        img_rgb_2 = morphological_operations(img_rgb)

        img_black_corners = highlight_black_corners(cv.cvtColor(img_rgb_2, cv.COLOR_BGR2GRAY))
        save_in_test(img_name+'_crop', img_black_corners)
        coords_1 = get_coordinates(img_black_corners, 0)
        crop = crop_image(img_rgb_2, coords_1)
        img_rgb = crop_image(img_rgb, coords_1)

        img_gray = cv.cvtColor(crop, cv.COLOR_BGR2GRAY)
        img_blue = crop[:,:,0]

        img_thresh_gray = threshold_image(img_gray)
        img_thresh_blue = threshold_image(img_blue)
        
        save_in_test(img_name+'_thresh_gray', img_thresh_gray)
        save_in_test(img_name+'_thresh_blue', img_thresh_blue)

        img_gray_no_corners = remove_corners(img_thresh_gray)
        img_blue_no_corners = remove_corners(img_thresh_blue)

        save_in_test(img_name+'_gray_no_corners', img_gray_no_corners)
        save_in_test(img_name+'_blue_no_corners', img_blue_no_corners)

        img_sum = add_gray_and_blue(img_gray_no_corners, img_blue_no_corners)
        img_clean = remove_minimal_blobs(img_sum)

        save_in_test(img_name+'_clean', img_clean)

        coords = get_coordinates(img_clean, 100)
        
        img_final = crop_image(img_rgb, coords)
        # print(coords, img_final.shape)
        max_coords = get_max_coords(max_coords, coords)

        save_image(img_name+'_final', img_final)
except Exception as e: 
    print(img_name)
    print(e)

In [233]:
# for img_name in images_name:
#     img_rgb = cv.imread('..\datasets\ISIC2017\ISIC-2017_Training_Data\{}.jpg'.format(img_name))
#     img = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
#     img = cv.GaussianBlur(img,(5,5),0)
#     _,img = cv.threshold(img,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
#     kernel = np.ones((10,10),np.uint8)
#     img = cv.erode(img,kernel,iterations = 1) 

#     img_blue = img_rgb[:,:,0]
#     img_blue = cv.GaussianBlur(img_blue,(11,11),0)
#     _,img_blue = cv.threshold(img_blue,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
#     kernel = np.ones((10,10),np.uint8)
#     img_blue = cv.erode(img_blue,kernel,iterations = 1)
    
#     y, x = img.shape

#     corners = {
#         'top_left': {
#             'x': [0,10],
#             'y': [0,10]
#         },
#         'top_right': {
#             'x': [0,10],
#             'y': [y - 11, y-1]
#         },
#         'bottom_left': {
#             'x': [x - 11, x-1],
#             'y': [0,10]
#         },
#         'bottom_right': {
#             'x': [x - 11, x-1],
#             'y': [y - 11, y-1]
#         },
#     }

#     count = 0
    
#     # remove white corners from image
#     for position in corners:
#         for x in range(corners[position]['x'][0], corners[position]['x'][1]):
#             for y in range(corners[position]['y'][0], corners[position]['y'][1]):
#                 if img[y,x] >= 255:
#                     cv.floodFill(img, None, (x,y), 0)
#                 if img_blue[y,x] >= 255:
#                     cv.floodFill(img_blue, None, (x,y), 0)
    
#     img_sum = img + img_blue
#     img_sum[img_sum > 125] = 255
#     img_sum[img_sum < 125] = 0

#     #remove minimal white blobs from images
#     contours, _ = cv.findContours(img_sum, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
#     img_clean = np.zeros_like(img_sum)
#     for contour in contours:
#         area = cv.contourArea(contour)
#         if area > 5000:
#             cv.fillPoly(img_clean, [contour], 255)

    
#     img_clean = cv.dilate(img_clean,kernel,iterations = 1)

#     contours, _ = cv.findContours(img_clean,cv.RETR_TREE,cv.CHAIN_APPROX_NONE)
    
#     t, b, r, l = [-1, 10000, -1, 10000]
    
#     for points in contours:
#         for p in points:
#             if t < p[0][1]:
#                 t = p[0][1]
#             if b > p[0][1]:
#                 b = p[0][1]
#             if l > p[0][0]:
#                 l = p[0][0]
#             if r < p[0][0]:
#                 r = p[0][0]

#     b = b - 100 if b - 100 > -1 else 0
#     t = t + 100 if t + 100 < y else y
#     l = l - 100 if l - 100 > -1 else 0
#     r = r + 100 if r + 100 < x else x
    
#     print(str(t), str(b), str(r), str(l))
                
#     img_final = cv.rectangle(img_rgb, (l,b), (r,t), (255, 0, 0), 1)

#     cv.imwrite('..\datasets\ISIC2017\\thresholding_experiments\methods_comparison\\'+img_name+'_method1.5.jpg', img_final)

In [234]:
coords = [0,1,2,3]


a, b, c, d = coords
print(a, b, c, d)

0 1 2 3
